In [ ]:
# Libraries required
import bluetooth
import pyttsx3
from joblib import  load

In [ ]:
ESP32_MAC_ADDRESS = '0C:8B:95:95:2A:FA'    # MAC address of ESP32's bluetooth

In [ ]:
#Load pretrained model
model_directory = 'C:\\Users\\sahil\\OneDrive\\Desktop\\model.joblib'
model=load(model_directory)

In [ ]:
# Start text to speach covterter
engine = pyttsx3.init()
print(engine.getProperty('rate'))
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id) #changing index changes voices but ony 0 and 1 are working here

In [ ]:
# Function to process received sensor data here
# Param: Data
# Ret: Nothing

def process_data(data):
    lis=data.split(",")
    lis = list(map(float, lis[0:5]))+list(map(float, lis[5:]))
    print(lis)
    lis=[lis]
    prediction=model.predict(lis)
    pyttsx3.speak(prediction[0])
    print(prediction)

In [ ]:
print("running")

def main():
    # Try to connect to bluetooth
    try:
        sock = bluetooth.BluetoothSocket(bluetooth.RFCOMM)
        sock.connect((ESP32_MAC_ADDRESS, 1))
        print(f"Connected to ESP32 with MAC address: {ESP32_MAC_ADDRESS}")
    except bluetooth.btcommon.BluetoothError as e:
        print(f"Error: {e}")
        return
    
    try:
        while True:
            try:
                # Read the size of the message (2 bytes)
                size_data = sock.recv(2)
                if not size_data:
                    print("Disconnected from ESP32.")
                    break
                
                # Ensure that size_data contains both bytes
                while len(size_data) < 2:
                    size_data += sock.recv(2 - len(size_data))

                # Convert bytes to an integer
                message_size = size_data[0] + (size_data[1] << 8)

                # Read the actual message based on the determined size
                data = sock.recv(message_size)
                if data:
                    try:
                        decoded_data = data.decode('utf-8').strip()
                        print("Received:", decoded_data)
                        process_data(decoded_data)        
                    except UnicodeDecodeError as e:
                        print("Error decoding data:", e)
                        print("Problematic bytes:", data.hex())
            except bluetooth.btcommon.BluetoothError as e:
                print(f"Bluetooth error: {e}")
                print("Disconnected from ESP32.")
                break
            
    except KeyboardInterrupt:
        print("\nExiting...")
    finally:
        sock.close()


if __name__ == "__main__":
    main()